In [1]:
import re
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

file_path = "/home/pan251/accel-sim-framework-pim/output.txt"
done_cycles = {}
start_cycles = {}

color_array = px.colors.qualitative.Plotly

freq = 0
last_done = 0

xbar_to_layer = {}


with open(file_path, "r") as file:
    for line in file:
        match = re.match(r"xbar (\d*) done layer, (\d*)", line)
        if match:
            xbar_id = int(match.group(1))
            cycle = int(match.group(2))
            if xbar_id not in done_cycles:
                done_cycles[xbar_id] = []
            done_cycles[xbar_id].append(cycle)
            last_done = cycle

        match = re.match(r"xbar (\d*) start compute, (\d*)", line)
        if match:
            xbar_id = int(match.group(1))
            cycle = int(match.group(2))
            if xbar_id not in start_cycles:
                start_cycles[xbar_id] = []
            start_cycles[xbar_id].append(cycle)

        match = re.match(r"-gpgpu_clock_domains (\d*.\d*):.*", line)
        if match:
            freq = float(match.group(1)) * 1e6

        match = re.match(r"layer (.*) mappped to xbar (\d*)", line)
        if match:
            layer = match.group(1)
            xbar = int(match.group(2))
            xbar_to_layer[xbar] = "layer " + layer


In [23]:

to_plot = pd.DataFrame(columns=["layer", "xbar", "start", "done", "duration", "color", "text"])
for xbar, starts in start_cycles.items():
    if xbar not in done_cycles:
        continue
    if (len(starts) != len(done_cycles[xbar])):
        done_cycles[xbar].append(last_done)
    assert(len(starts) == len(done_cycles[xbar]))
    dones = done_cycles[xbar]
    for index, start in enumerate(starts):
        start = start / freq * 1e3
        done = dones[index] / freq * 1e3
        duration = (done - start)
        # format
        # duration = "{:.5f}".format(duration)
        print(duration)
        
        to_plot = pd.concat([to_plot, pd.DataFrame({
            "layer": [xbar_to_layer[xbar]],
            "xbar": [str(xbar)], 
            "start": [start], 
            "duration": [duration], 
            "done": [done], 
            "color": [color_array[index % len(color_array)]],
            "text": [f"{duration:.2f} ms"]
            })])
        
fig = go.Figure()
fig.add_trace(go.Bar(
    x = to_plot["duration"],
    y = [to_plot["layer"],"xbar " + to_plot["xbar"]],
    orientation = "h",
    base = to_plot["start"],
    marker_color = to_plot["color"],
    # add text duration
    text = to_plot["text"],
)
)

# change size
fig.update_layout(
    autosize=False,
    width=to_plot["duration"].max() * 50,
    height=len(to_plot["layer"].unique()) * 100,
    margin=dict(l=0, r=0, t=0, b=0)
)

# change x range
fig.update_xaxes(range=[0, to_plot["start"].max() * 1.1])

fig.show()
# write jpg
fig.write_image("pip.jpg", width=1920, height=1080, scale=2)
fig.write_html("./pip.html")

14.674276501766784
14.674274734982333
14.67427473498233
14.674274734982333
14.674274734982333
14.674274734982319
14.674274734982333
14.674274734982333
14.674274734982319
14.674274734982333
14.674274734982333
14.674274734982333
14.674274734982333
14.674274734982333
14.674274734982362
14.674274734982305
14.674274734982305
14.674274734982305
14.674274734982362
14.674274734982362
14.674276501766826
14.674274734982362
14.674276501766826
14.674274734982362
14.674276501766826
14.674274734982362
14.674274734982362
14.674274734982305
14.674274734982305
14.674274734982362
14.674274734982362
14.674274734982362
14.674274734982305
14.674274734982248
14.674274734982362
14.674274734982305
14.674274734982419
14.674274734982419
14.674274734982419
14.674274734982419
14.674274734982305
14.674274734982305
14.674274734982305
14.674274734982305
14.67429946996458
14.674274734982305
14.674274734982419
14.674274734982419
14.674274734982419
14.674274734982419
14.674274734982305
14.674274734982419
14.67427473498

In [1]:
import onnx

model = onnx.load("/home/pan251/accel-sim-framework-pim/resnet50.onnx")

with open("/home/pan251/accel-sim-framework-pim/serialized.model", "wb") as f:
  f.write(model.graph.SerializeToString())


In [12]:
to_plot["duration"]

0    14.674277
0    14.674275
0    14.674275
0    14.674275
0    14.674275
       ...    
0    86.071648
0    54.374099
0    78.589089
0    54.427327
0    20.335157
Name: duration, Length: 964, dtype: object